In [5]:
import os

In [6]:
os.chdir("../")

In [7]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class CallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [8]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [9]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    
    def get_callback_config(self) -> CallbacksConfig:
        config = self.config.callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        callback_config = CallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return callback_config

In [10]:
ConfigurationManager().get_callback_config()

[2023-12-30 23:36:49,772: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-12-30 23:36:49,774: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-30 23:36:49,775: INFO: common: created directory at: artifacts]
[2023-12-30 23:36:49,776: INFO: common: created directory at: artifacts/callbacks/checkpoint_dir]
[2023-12-30 23:36:49,777: INFO: common: created directory at: artifacts/callbacks/tensorboard_log_dir]


CallbacksConfig(root_dir=PosixPath('artifacts/callbacks'), tensorboard_root_log_dir=PosixPath('artifacts/callbacks/tensorboard_log_dir'), checkpoint_model_filepath=PosixPath('artifacts/callbacks/checkpoint_dir/model.keras'))

### component

In [17]:
import os
import tensorflow as tf
import keras
import time
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import pandas as pd

2023-12-30 23:38:47.514182: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-30 23:38:47.560761: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-30 23:38:47.560786: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-30 23:38:47.561676: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-30 23:38:47.566955: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-30 23:38:47.567602: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [18]:
class PrepareCallback:
    def __init__(self, config: CallbacksConfig):
        self.config = config

    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=str(self.config.checkpoint_model_filepath),
            save_best_only=True,
            monitor='val_accuracy',
            mode='max'
        )


    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

In [19]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()
    
except Exception as e:
    raise e

[2023-12-30 23:38:49,388: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-12-30 23:38:49,393: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-30 23:38:49,394: INFO: common: created directory at: artifacts]
[2023-12-30 23:38:49,395: INFO: common: created directory at: artifacts/callbacks/checkpoint_dir]
[2023-12-30 23:38:49,395: INFO: common: created directory at: artifacts/callbacks/tensorboard_log_dir]


In [20]:
callback_list